<a href="https://colab.research.google.com/github/shilz1007/shilz1007/blob/main/DailyDialog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import codecs
import csv
import os
import unicodedata
import re
import itertools
import os
import sys
import random
import unicodedata
import re
import argparse
import numpy as np
import pandas as pd
from tqdm import tqdm
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import sentence_bleu
#from transformers import GPT2Tokenizer,TFGPT2LMHeadModel 
from argparse import Namespace
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [2]:
path1 = '/content/drive/MyDrive/Daily-Dialog/dialogues_train.txt'
path2 = '/content/drive/MyDrive/Daily-Dialog/dialogues_act_train.txt'
path3 = '/content/drive/MyDrive/Daily-Dialog/dialogues_test.txt'
path4 = '/content/drive/MyDrive/Daily-Dialog/dialogues_act_test.txt'
path5 = '/content/drive/MyDrive/Daily-Dialog/dialogues_validation.txt'
path6 = '/content/drive/MyDrive/Daily-Dialog/dialogues_act_validation.txt'

In [ ]:
#corpus_name = "dailydialog"
#corpus = os.path.join("data", corpus_name)

In [3]:
def print_lines(file,n=10):
  with open(file,'r',encoding='utf-8') as datafile:
    lines = datafile.readlines()
  for line in lines[:n]:
    print(line)

In [3]:
def load_lines(input_file):
  conversations = []
  with open(input_file,'r',encoding='utf-8') as f:
    for line in f:
      values = line.split("__eou__")
      conversations.append(values)
  return conversations   


In [4]:
def load_dialog(acts):
  dialog =[]
  with open(acts,'r',encoding='utf-8') as f:
    for line in f:
      for word in line.split():
        #for i in range(len(line) - 1):
        dialog.append(word)
        #  input_act = line[i]
        #  target_act = line[i + 1]
        #  print('input_act',input_act)
        #  print('target_act',target_act)
          #print(word)
        #  #dialog.append([str(first),str(second)]) 
        #  dialog.append([input_act,target_act])
  return dialog    

In [6]:
def extract_sentence_pairs(conversations):
  conv_pairs = []
  for conversation in conversations:
    for i in range(len(conversation) - 1):
      input_line = conversation[i].strip()
      target_line = conversation[i+1].strip()
      if input_line and target_line:
        conv_pairs.append([input_line,target_line])
  return conv_pairs      

In [5]:
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD',s)
                 if unicodedata.category(c) != 'Mn')

In [6]:
def normalize_string(s):
    s = unicode_to_ascii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    s = re.sub(r"\s+([.!?])", r"\1", s)
    s = s.rstrip(".,!?")
    return s

In [3]:
#def normalize_string(s):
#    s = s.lower().strip()
#    s = re.sub(r"([.!?])", r" \1", s)
#    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
#    s = re.sub(r"\s+", r" ", s).rstrip(".,!?")
#    return s

In [7]:
def format_data(source,target,acts,type=None):
  data_file = target
  delimiter = '\t'
  delimiter = str(codecs.decode(delimiter,"unicode_escape"))

  conversations = load_lines(source)
  dialog_acts = load_dialog(acts)
  #print(dialog_acts[0])
  #print(conversations[0])
  conv_pairs = []
  for conversation in conversations:
    for i in range(len(conversation) - 1):
      input_line = conversation[i].strip()
      target_line = conversation[i+1].strip()
      if input_line and target_line:
        conv_pairs.append([input_line,target_line])
  print(conv_pairs[0]) 
  #merging dialog and dialog act     
  conv_pairs_with_dialog_act = []
  for i in range(len(conv_pairs)):
    input_line = conv_pairs[i][0]
    target_line = conv_pairs[i][1]
    input_act = dialog_acts[i]
    target_act = dialog_acts[i+1]

    conv_pairs_with_dialog_act.append([input_line,target_line,input_act,target_act])

  print(conv_pairs_with_dialog_act[0])  
  #conv_pairs_normalize = []
  #for i in range(len(conv_pairs_with_dialog_act)):
  #   input_line = normalize_string(conv_pairs_with_dialog_act[i][0])
  #   target_line = normalize_string(conv_pairs_with_dialog_act[i][1])
  #   input_act = normalize_string(conv_pairs_with_dialog_act[i])
  #   target_act = normalize_string(conv_pairs_with_dialog_act[i+1])
  #   conv_pairs_normalize.append([input_line,target_line,input_act,target_act])
  #print(conv_pairs_normalize[0])
  #Writing out the combined dataset 
  print('\n writing a new file')
  with open(data_file,'w',encoding='utf-8') as output_file:
    writer = csv.writer(output_file,delimiter=delimiter,lineterminator='\n')
    for pair in conv_pairs_with_dialog_act:
      writer.writerow(pair)



  

  
  
  


In [15]:
format_data(source=path1,target ="/content/drive/MyDrive/Daily-Dialog/formatted_dialogues_train.txt" ,acts=path2)


['Say , Jim , how about going for a few beers after dinner ?', 'You know that is tempting but is really not good for our fitness .']
['Say , Jim , how about going for a few beers after dinner ?', 'You know that is tempting but is really not good for our fitness .', '3', '4']

 writing a new file


In [16]:
format_data(source=path5,target ="/content/drive/MyDrive/Daily-Dialog/formatted_dialogues_valid.txt",acts=path6)

['Good morning , sir . Is there a bank near here ?', 'There is one . 5 blocks away from here ?']
['Good morning , sir . Is there a bank near here ?', 'There is one . 5 blocks away from here ?', '2', '1']

 writing a new file


In [8]:
format_data(source=path3,target ="/content/drive/MyDrive/Daily-Dialog/formatted_dialogues_test.txt",acts=path4)

['Hey man , you wanna buy some weed ?', 'Some what ?']
['Hey man , you wanna buy some weed ?', 'Some what ?', '3', '2']

 writing a new file


In [9]:
#reading the formatted file
train_file = '/content/drive/MyDrive/Daily-Dialog/formatted_dialogues_train.txt'
valid_file = '/content/drive/MyDrive/Daily-Dialog/formatted_dialogues_valid.txt'
test_file = '/content/drive/MyDrive/Daily-Dialog/formatted_dialogues_test.txt'





In [18]:
def read_input_file(input_file):
  n = 10
  with open(input_file,'r',encoding='utf-8') as datafile:
     lines = datafile.readlines()
     for line in lines[:n]:
       print(line)

In [19]:
input_file = read_input_file(train_file)

Say , Jim , how about going for a few beers after dinner ?	You know that is tempting but is really not good for our fitness .	3	4

You know that is tempting but is really not good for our fitness .	What do you mean ? It will help us to relax .	4	2

What do you mean ? It will help us to relax .	Do you really think so ? I don't . It will just make us fat and act silly . Remember last time ?	2	2

Do you really think so ? I don't . It will just make us fat and act silly . Remember last time ?	I guess you are right.But what shall we do ? I don't feel like sitting at home .	2	2

I guess you are right.But what shall we do ? I don't feel like sitting at home .	I suggest a walk over to the gym where we can play singsong and meet some of our friends .	2	3

I suggest a walk over to the gym where we can play singsong and meet some of our friends .	That's a good idea . I hear Mary and Sally often go there to play pingpong.Perhaps we can make a foursome with them .	3	4

That's a good idea . I hear M

In [10]:
#preprocess data 
def preprocess_data(infile,target):

  # add generic responses
  # change all the words to lower character.
  # remove stop words and special character
  # format for unicode to ascii 
  delimiter = '\t'
  output = target

  #if args.infile == train_file:
  #   output = os.path.join(args.data_path, "train.txt")
  #else:   
  #   output = os.path.join(args.data_path, "valid.txt") 
  
  
  #path = '/content/drive/MyDrive/Daily-Dialog'
  new_pair = []

  with open(infile,'r',encoding='utf-8') as f:
    for line in f:
      try:
        input_line,target_line,dialog_act_input,dialog_act_target = line.strip().split('\t')
      except ValueError:
        print('skipping invalid line ',line)  
      new_pair.append([input_line.strip(),target_line.strip(),dialog_act_input.strip(),dialog_act_target.strip()])

  generic_responses = [
    "I see.", 
    "That's interesting.", 
    "Can you tell me more about that?", 
    "I'm not sure I understand. Can you clarify?", 
    "Thank you for sharing.", 
    "That's a good point.", 
    "I agree.", 
    "I disagree.", 
    "Let's change the topic.", 
    "That reminds me of something else.",
    "I'm sorry, I don't have an answer for that.",
    "I don't know what to say.",
    ]

  p_generic_responses = 0.1

  new_conv_pair = []

  for input_line,target_line,dialog_act_input,dialog_act_target in new_pair:

    new_conv_pair.append([input_line,target_line,dialog_act_input,dialog_act_target])
    if random.random() < p_generic_responses:
      generic_response = random.choice(generic_responses)
      new_conv_pair.append([input_line,generic_response,dialog_act_input,'5'])

  print(new_conv_pair[0]) 
  #normalize data
  normalized_conv_pair = []

  for pair in new_conv_pair:
    input_line,target_line,dialog_act_input,dialog_act_target = pair

    #normalize the input line and target line
    input_line = normalize_string(input_line)
    target_line = normalize_string(target_line)

    #append the normalized pair back to the list

    normalized_pair = [input_line,target_line,dialog_act_input,dialog_act_target]
    normalized_conv_pair.append(normalized_pair)
  print(normalized_conv_pair[0]) 

  

  #for pair in normalized_conv_pair:
  #  input_line,target_line,dialog_act_input,dialog_act_target = pair
  #  if dialog_act_target == '5':
  #    print(pair)
  
  #write output file 
  print('\n writing a new file')
  with open(output,'w',encoding='utf-8') as output_file:
    writer = csv.writer(output_file,delimiter=delimiter,lineterminator='\n')
    for pair in normalized_conv_pair:
      writer.writerow(pair)




    







In [38]:
target = '/content/drive/MyDrive/Daily-Dialog/train.txt'

In [39]:
infile = preprocess_data(train_file,target)

['Say , Jim , how about going for a few beers after dinner ?', 'You know that is tempting but is really not good for our fitness .', '3', '4']
['say jim how about going for a few beers after dinner', 'you know that is tempting but is really not good for our fitness', '3', '4']

 writing a new file


In [40]:
target = '/content/drive/MyDrive/Daily-Dialog/valid.txt'

In [41]:
validfile = preprocess_data(valid_file,target)

['Good morning , sir . Is there a bank near here ?', 'There is one . 5 blocks away from here ?', '2', '1']
['good morning sir. is there a bank near here', 'there is one. blocks away from here', '2', '1']

 writing a new file


In [12]:
target = '/content/drive/MyDrive/Daily-Dialog/test.txt'
testfile = preprocess_data(test_file,target)

['Hey man , you wanna buy some weed ?', 'Some what ?', '3', '2']
['hey man you wanna buy some weed', 'some what', '3', '2']

 writing a new file
